In [1]:
#Install dependencies if not already installed
!pip install tensorflow'>=2.3' torch'>=1.6' fastestimator ~/repos/fledge

Processing /home/itrushkin/repos/fledge


  Created wheel for fledge: filename=fledge-0.20-py3-none-any.whl size=168516 sha256=e3b5ff851f4e85f133fea05d6ce19ee20928a4ab0610cb552ad1f25dcfe82648
  Stored in directory: /tmp/pip-ephem-wheel-cache-xlqca__t/wheels/3a/30/9d/a88101effd240e4a8721301bebf95a245eaf4904b55b7ae530
Successfully built fledge
  Attempting uninstall: fledge
    Found existing installation: fledge 0.20
    Uninstalling fledge-0.20:
      Successfully uninstalled fledge-0.20


In [2]:
import fastestimator as fe
from fastestimator.architecture.pytorch import LeNet
from fastestimator.backend import to_tensor, argmax
from fastestimator.dataset.data import cifar10
from fastestimator.op.numpyop.meta import Sometimes
from fastestimator.op.numpyop.multivariate import HorizontalFlip
from fastestimator.op.numpyop.univariate import CoarseDropout, Normalize, Onehot
from fastestimator.op.tensorop import Average
from fastestimator.op.tensorop.gradient import Watch, FGSM
from fastestimator.op.tensorop.loss import CrossEntropy
from fastestimator.op.tensorop.model import ModelOp, UpdateOp
from fastestimator.trace.io import BestModelSaver
from fastestimator.trace.metric import Accuracy
from fastestimator.util import ImgData, to_number
from fledge.native.fastestimator import FederatedFastEstimator
from fastestimator.dataset.data import cifar10
from fastestimator.trace.adapt import LRScheduler
from fastestimator.op.numpyop.univariate import Normalize, ChannelTranspose

batch_size=128

train_data, eval_data = cifar10.load_data()
test_data = eval_data.split(0.5)

pipeline = fe.Pipeline(train_data=train_data,
                     eval_data=eval_data,
                     test_data=test_data,
                     batch_size=batch_size,
                     ops=[Normalize(inputs="x", outputs="x",
                             mean=(0.4914, 0.4822, 0.4465), 
                             std=(0.2471, 0.2435, 0.2616)),
                          ChannelTranspose(inputs="x", outputs="x")])
model = fe.build(model_fn=lambda: LeNet(input_shape=(3, 32, 32)), \
                         optimizer_fn="adam", model_name="adv_model")

network = fe.Network(ops=[
                Watch(inputs="x"),
                ModelOp(model=model, inputs="x", outputs="y_pred"),
                CrossEntropy(inputs=("y_pred", "y"), outputs="base_ce"),
                FGSM(data="x", loss="base_ce", outputs="x_adverse", epsilon=0.04),
                ModelOp(model=model, inputs="x_adverse", outputs="y_pred_adv"),
                CrossEntropy(inputs=("y_pred_adv", "y"), outputs="adv_ce"),
                Average(inputs=("base_ce", "adv_ce"), outputs="avg_ce"),
                UpdateOp(model=model, loss_name="avg_ce")
            ])
def lr_schedule(step):
    lr = 1 / (100 * step)
    return lr

estimator = fe.Estimator(pipeline=pipeline,
                         network=network,
                         epochs=1,
                         traces=[Accuracy(true_key="y", pred_key="y_pred", output_name="clean_accuracy"),
                                Accuracy(true_key="y", pred_key="y_pred_adv", output_name="adversarial_accuracy"),
                                LRScheduler(model=model, lr_fn=lr_schedule)],
                         max_train_steps_per_epoch=None,
                         max_eval_steps_per_epoch=None,
                         monitor_names=["base_ce", "adv_ce"],
                         log_steps=50)
fledge_estimator = FederatedFastEstimator(estimator, rounds=10)
model=fledge_estimator.fit()

Creating Workspace Directories
Creating Workspace Certifications
Creating Workspace Templates

New workspace directory structure:
workspace
├── code
│   ├── keras_cnn.py
│   ├── mnist_utils.py
│   ├── __init__.py
│   └── tfmnist_inmemory.py
├── agg_to_col_two_signed_cert.zip
├── plan
│   ├── cols.yaml
│   ├── data.yaml
│   ├── defaults
│   │   ├── tasks_keras.yaml
│   │   ├── tasks_tensorflow.yaml
│   │   ├── tasks_fast_estimator.yaml
│   │   ├── network.yaml
│   │   ├── collaborator.yaml
│   │   ├── assigner.yaml
│   │   ├── tasks_torch.yaml
│   │   ├── aggregator.yaml
│   │   ├── task_runner.yaml
│   │   └── data_loader.yaml
│   └── plan.yaml
├── requirements.txt
├── logs
├── agg_to_col_one_signed_cert.zip
├── cert
│   ├── config
│   │   ├── client.conf
│   │   ├── signing-ca.conf
│   │   ├── server.conf
│   │   └── root-ca.conf
│   ├── client
│   │   ├── col_one.crt
│   │   ├── col_two.csr
│   │   ├── col_two.crt
│   │   ├── col_two.key
│   │   ├── col_one.csr
│   │   └── col_one.ke

traces=[<fastestimator.trace.metric.accuracy.Accuracy object at 0x7fc005c8c2e8>, <fastestimator.trace.metric.accuracy.Accuracy object at 0x7fc005c95240>, <fastestimator.trace.adapt.lr_scheduler.LRScheduler object at 0x7fc005c956d8>]


traces=[<fastestimator.trace.metric.accuracy.Accuracy object at 0x7fc005c8c2e8>, <fastestimator.trace.metric.accuracy.Accuracy object at 0x7fc005c95240>, <fastestimator.trace.adapt.lr_scheduler.LRScheduler object at 0x7fc005c956d8>]


traces=[<fastestimator.trace.metric.accuracy.Accuracy object at 0x7fc005c8c2e8>, <fastestimator.trace.metric.accuracy.Accuracy object at 0x7fc005c95240>, <fastestimator.trace.adapt.lr_scheduler.LRScheduler object at 0x7fc005c956d8>]


/home/itrushkin/.virtualenvs/empty/lib/python3.6/site-packages/fledge/federated/task/runner_pt.py:172: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  new_state[k] = pt.from_numpy(tensor_dict.pop(k)).to(device)


...

    ______           __  ______     __  _                 __            
   / ____/___ ______/ /_/ ____/____/ /_(_)___ ___  ____ _/ /_____  _____
  / /_  / __ `/ ___/ __/ __/ / ___/ __/ / __ `__ \/ __ `/ __/ __ \/ ___/
 / __/ / /_/ (__  ) /_/ /___(__  ) /_/ / / / / / / /_/ / /_/ /_/ / /    
/_/    \__,_/____/\__/_____/____/\__/_/_/ /_/ /_/\__,_/\__/\____/_/     
                                                                        

FastEstimator-Warn: No ModelSaver Trace detected. Models will not be saved.
FastEstimator-Start: step: 882; num_device: 3; logging_interval: 50; 
FastEstimator-Eval: step: 882; epoch: 9; avg_ce: 1.9635458; base_ce: 1.8905987; adv_ce: 2.0364928; clean_accuracy: 0.3248; adversarial_accuracy: 0.2552; 
FastEstimator-Train: step: 900; avg_ce: 1.9675782; base_ce: 1.8962873; adv_ce: 2.038869; steps/sec: 93.18; adv_model_lr: 1.1111111e-05; 
FastEstimator-Train: step: 950; avg_ce: 1.9204886; base_ce: 1.842888; adv_ce: 1.9980892; steps/sec: 32.38; adv_model_lr: 1.0

FastEstimator-Test: step: 980; epoch: 11; clean_accuracy: 0.3096; adversarial_accuracy: 0.2384; 


FastEstimator-Test: step: 882; epoch: 10; clean_accuracy: 0.332; adversarial_accuracy: 0.2576; 


    ______           __  ______     __  _                 __            
   / ____/___ ______/ /_/ ____/____/ /_(_)___ ___  ____ _/ /_____  _____
  / /_  / __ `/ ___/ __/ __/ / ___/ __/ / __ `__ \/ __ `/ __/ __ \/ ___/
 / __/ / /_/ (__  ) /_/ /___(__  ) /_/ / / / / / / /_/ / /_/ /_/ / /    
/_/    \__,_/____/\__/_____/____/\__/_/_/ /_/ /_/\__,_/\__/\____/_/     
                                                                        

FastEstimator-Warn: No ModelSaver Trace detected. Models will not be saved.
FastEstimator-Start: step: 882; num_device: 3; logging_interval: 50; 
FastEstimator-Eval: step: 882; epoch: 9; avg_ce: 1.9070349; base_ce: 1.8382574; adv_ce: 1.9758126; clean_accuracy: 0.3264; adversarial_accuracy: 0.264; 
FastEstimator-Train: step: 900; avg_ce: 1.9324747; base_ce: 1.8604195; adv_ce: 2.00453; steps/sec: 94.83; adv_model_lr: 1.1111111e-05; 
FastEstimator-Train: step: 950; avg_ce: 1.8576719; base_ce: 1.7990401; adv_ce: 1.9163036; steps/sec: 32.63; adv_model_lr: 1.05

FastEstimator-Test: step: 980; epoch: 11; clean_accuracy: 0.3328; adversarial_accuracy: 0.2624; 


In [4]:
model.state_dict()

OrderedDict([('module.conv1.weight',
              tensor([[[[ 0.0480, -0.1234,  0.0084],
                        [-0.0746, -0.0893,  0.1434],
                        [-0.2028, -0.1373, -0.0363]],
              
                       [[ 0.1610, -0.1625,  0.0500],
                        [ 0.1089,  0.1591, -0.1542],
                        [ 0.0251,  0.0867, -0.0326]],
              
                       [[ 0.1283,  0.0837, -0.1000],
                        [-0.0044,  0.0681, -0.0939],
                        [-0.0952, -0.2005, -0.0609]]],
              
              
                      [[[ 0.0842, -0.1372, -0.1116],
                        [-0.0936, -0.1285,  0.1557],
                        [-0.1403,  0.1306, -0.0184]],
              
                       [[ 0.0373, -0.0589, -0.0689],
                        [ 0.0609, -0.0477, -0.0254],
                        [ 0.1200,  0.1611,  0.1638]],
              
                       [[-0.0595, -0.0828, -0.0330],
                   